In [46]:
import requests
import json
import pandas as pd

pd.set_option('display.max_columns', None)

In [311]:
# https://github.com/xivapi/ffxiv-datamining/blob/master/csv/Recipe.csv
recipes = pd.read_csv('./Recipe.csv')
# https://github.com/xivapi/ffxiv-datamining/blob/master/csv/Item.csv
items = pd.read_csv('./Item.csv')


with open('./item_dictionary.json','r') as f:
    item_dictionary = json.load(f)

In [338]:
def get_curr_prices(item_code, dc='Aether'):
    query = f'https://universalis.app/api/v2/{dc}/{item_code}'

    response = requests.get(query)

    try:
        assert(response.status_code == 200)
        listings = pd.DataFrame(response.json()['listings'])
        meta = 
        return pd.DataFrame(response.json()['listings']), 
        
    except AssertionError:
        print(f'{response.status_code}\n{response.content}')

In [216]:
def get_item_code(item_name):
    query = f'https://xivapi.com/search?string={item_name}'

    response = requests.get(query)

    try:
        assert(response.status_code == 200)
        item_code = response.json()['Results'][0]['ID']
        return item_code 
    except AssertionError:
        print(f'{response.status_code}\n{response.content}')

In [271]:
def get_recipe(item_code, num_needed):
    try:
        ingredients = {recipes.loc[recipes['Item{Result}'] == item_code,f'Item{{Ingredient}}[{i}]'].iloc[0]:
                           recipes.loc[recipes['Item{Result}'] == item_code,f'Amount{{Ingredient}}[{i}]'].iloc[0] 
                               for i in range(10) 
                               if recipes.loc[recipes['Item{Result}'] == item_code,f'Amount{{Ingredient}}[{i}]'].iloc[0] > 0}
        num_produced = recipes.loc[recipes['Item{Result}'] == item_code, 'Amount{Result}'].iloc[0]

        total_ingredients = {k: (v * num_needed)/num_produced for k,v in ingredients.items()}
        return total_ingredients, num_produced
    except IndexError:
        return (None,None)

In [329]:
def get_crafting_tree(item_code, num_needed = 1):
    item_name = items.loc[item_code,'Singular']
    recipe, recipe_yields = get_recipe(item_code, num_needed)
    
    output = {
        'name': item_name.title(),
        'id': item_code,
        'number_needed': num_needed
    }

    if recipe is None:
        return output
    else:
        output.update({'recipe': [get_crafting_tree(item, num) for item, num in recipe.items()],
                       'recipe_yields': recipe_yields})

    return output

In [221]:
# item_names = ['Ashen Alumen','Calf Leather','Calfskin Rider\'s Bottoms','Calfskin Rider\'s Cap','Calfskin Rider\'s Gloves','Calfskin Rider\'s Jacket','Calfskin Rider\'s Shoes','Caprice Fleece','Dimythrite Ore','Duskblooms','Duskcourt Cloth','Dwarven Cotton','Dwarven Cotton Boll','Dwarven Cotton Thread','Dwarven Mythril Ingot','Ethereal Cocoon','Ethereal Silk','Manasilver Nugget','Manasilver Sand','Manzasiri Hair','Megalania Leather','Megalania Skin','Multifaceted Cotton Cloth','Mythrite Ore','Rebel Coat','Shell Leather','Silver Ore']
# item_dictionary = {item_name: get_item_code(item_name) for item_name in item_names}
# with open('./item_dictionary.json','w') as f:
#     f.write(json.dumps(item_dictionary))

In [ ]:
# current price of every item
# mapping of crafting recipe and # of needed ingredients
# recent sale volume
# END GOAL: most profitable craftables

In [332]:
item_dictionary

{'Ashen Alumen': 29976,
 'Calf Leather': 33027,
 "Calfskin Rider's Bottoms": 32801,
 "Calfskin Rider's Cap": 32798,
 "Calfskin Rider's Gloves": 32800,
 "Calfskin Rider's Jacket": 32799,
 "Calfskin Rider's Shoes": 32802,
 'Caprice Fleece': 29975,
 'Dimythrite Ore': 27703,
 'Duskblooms': 29974,
 'Duskcourt Cloth': 29961,
 'Dwarven Cotton': 26289,
 'Dwarven Cotton Boll': 27759,
 'Dwarven Cotton Thread': 27758,
 'Dwarven Mythril Ingot': 27714,
 'Ethereal Cocoon': 27761,
 'Ethereal Silk': 27760,
 'Manasilver Nugget': 27710,
 'Manasilver Sand': 27699,
 'Manzasiri Hair': 19986,
 'Megalania Leather': 29962,
 'Megalania Skin': 29977,
 'Multifaceted Cotton Cloth': 27762,
 'Mythrite Ore': 12534,
 'Rebel Coat': 28590,
 'Shell Leather': 28905,
 'Silver Ore': 5113}

In [334]:
get_crafting_tree(32800)

{'name': "Pair Of Calfskin Rider'S Gloves",
 'id': 32800,
 'number_needed': 1,
 'recipe': [{'name': 'Circle Of Calf Leather',
   'id': 33027,
   'number_needed': 1.0},
  {'name': 'Bolt Of Duskcourt Cloth',
   'id': 29961,
   'number_needed': 3.0,
   'recipe': [{'name': 'Clump Of Duskblooms',
     'id': 29974,
     'number_needed': 12.0},
    {'name': 'Basket Of Caprice Fleece', 'id': 29975, 'number_needed': 6.0},
    {'name': 'Lightning Cluster', 'id': 18, 'number_needed': 6.0},
    {'name': 'Wind Cluster', 'id': 16, 'number_needed': 6.0}],
   'recipe_yields': 1},
  {'name': 'Earth Crystal', 'id': 11, 'number_needed': 7.0},
  {'name': 'Wind Crystal', 'id': 10, 'number_needed': 7.0}],
 'recipe_yields': 1}

In [335]:
get_crafting_tree(32799)

{'name': "Calfskin Rider'S Jacket",
 'id': 32799,
 'number_needed': 1,
 'recipe': [{'name': 'Circle Of Calf Leather',
   'id': 33027,
   'number_needed': 2.0},
  {'name': 'Circle Of Megalania Leather',
   'id': 29962,
   'number_needed': 3.0,
   'recipe': [{'name': 'Megalania Skin', 'id': 29977, 'number_needed': 6.0},
    {'name': 'Pinch Of Ashen Alumen', 'id': 29976, 'number_needed': 12.0},
    {'name': 'Earth Cluster', 'id': 17, 'number_needed': 6.0},
    {'name': 'Wind Cluster', 'id': 16, 'number_needed': 6.0}],
   'recipe_yields': 1},
  {'name': 'Dwarven Mythril Ingot',
   'id': 27714,
   'number_needed': 1.0,
   'recipe': [{'name': 'Chunk Of Dimythrite Ore',
     'id': 27703,
     'number_needed': 4.0},
    {'name': 'Chunk Of Mythrite Ore', 'id': 12534, 'number_needed': 1.0},
    {'name': 'Fire Crystal', 'id': 8, 'number_needed': 7.0}],
   'recipe_yields': 1},
  {'name': 'Earth Crystal', 'id': 11, 'number_needed': 7.0},
  {'name': 'Wind Crystal', 'id': 10, 'number_needed': 7.0}],


In [354]:
jacket_prices = get_curr_prices(32799)

In [341]:
jacket_prices.keys()

dict_keys(['itemID', 'lastUploadTime', 'listings', 'recentHistory', 'dcName', 'currentAveragePrice', 'currentAveragePriceNQ', 'currentAveragePriceHQ', 'regularSaleVelocity', 'nqSaleVelocity', 'hqSaleVelocity', 'averagePrice', 'averagePriceNQ', 'averagePriceHQ', 'minPrice', 'minPriceNQ', 'minPriceHQ', 'maxPrice', 'maxPriceNQ', 'maxPriceHQ', 'stackSizeHistogram', 'stackSizeHistogramNQ', 'stackSizeHistogramHQ', 'worldUploadTimes', 'listingsCount', 'recentHistoryCount', 'unitsForSale', 'unitsSold'])

In [356]:
jacket_prices['averagePriceHQ']

1020441.8

In [355]:
pd.DataFrame(jacket_prices['listings'])

lastReviewTime  pricePerUnit  quantity  stainID   worldName  worldID  \
0       1709330950        900000         1        0      Jenova       40   
1       1709342133        948000         1        0  Sargatanas       99   
2       1709342133        948000         1        0  Sargatanas       99   
3       1709337884        948999         1        0  Sargatanas       99   
4       1709318515        950000         1        0  Sargatanas       99   
..             ...           ...       ...      ...         ...      ...   
78      1709342491       2000000         1        0   Gilgamesh       63   
79      1709346806       6500000         1        0      Faerie       54   
80      1709346806       6500000         1        0      Faerie       54   
81      1709346806       6500000         1        0      Faerie       54   
82      1709343529     999999999         1        0  Sargatanas       99   

   creatorName                                          creatorID     hq  \
0                                                            None  False   
1               6999d122b768e0ff9177996bb5d04caa9ea061899c590f...   True   
2               6999d122b768e0ff9177996bb5d04caa9ea061899c590f...   True   
3               871f0c610b97f5cd308d6875bc2ba62ef03871c0933055...   True   
4               3890f5a5a3c5a570a6f426b082d9046140196c25759fa3...   True   
..         ...                                                ...    ...   
78              d630917c63ac58bc4e01791837a41c48ba0e72d32acd54...   True   
79                                                           None  False   
80                                                           None  False   
81              9d309c21e6a01076809829c9b7fc6a645d410daab722b6...   True   
82              cea8562bbb426623d6fc2933d7bd1b933e71174b575830...  False   

    isCrafted                                          listingID materia  \
0       False  81800e9da4d907a087eebc1dcfb82d4e28c85440fc1c3c...      []   
1        True  f408af6eae6368aa8c92908d640471e85b632628c30d02...      []   
2        True  9e0363ee8c8c1f83fbd1dec3fb50730ce772df6a45aedc...      []   
3        True  6a1825c1a5326345a083afa47f5a8860465b82361eeed8...      []   
4        True  bb9c0eb20f49d484a3669022f2911580df397a7998310a...      []   
..        ...                                                ...     ...   
78       True  24222dc48a66c488f8bf7424cb7d04156b4a1f2cecf43d...      []   
79      False  2da70d456827f7082a3ed522d4152981fdecbb27a1ccbb...      []   
80      False  1e60d3ff5b3a60dc43828e15a0ff3f1d9dc31368d511c2...      []   
81       True  a462ddda98df2ced3a87774f3b813fcd91c0da5c82c637...      []   
82       True  d0f608c03702bccb3a1209551497f8872025255e4289fb...      []   

    onMannequin  retainerCity  \
0         False             1   
1         False            10   
2         False            10   
3         False             2   
4         False             4   
..          ...           ...   
78        False             1   
79        False             1   
80        False             1   
81        False             1   
82        False             2   

                                           retainerID     retainerName  \
0   4351f7dee2623e4b93ac41b31beef929a3b44041b53f5d...           Faasda   
1   70e6467bd0ea7b290b74c1ed6dac1fc0562aacd2616cce...    Mrs-honey-pot   
2   70e6467bd0ea7b290b74c1ed6dac1fc0562aacd2616cce...    Mrs-honey-pot   
3   747b54616f463b10196e2373cfda2e3e410bdd5baa4eec...  Aerynssidepiece   
4   e23b75aa76c4b3ff47840ec3da333214a138bb33d9d164...     Kyoko-sakura   
..                                                ...              ...   
78  c8d66da3a45bc7e737b8f3749cb66164c0b4de02b0b098...            Poggy   
79  a6419fd3a501f99c6d0451846fe474f6222e5676e04bcf...           Elisay   
80  a6419fd3a501f99c6d0451846fe474f6222e5676e04bcf...           Elisay   
81  a6419fd3a501f99c6d0451846fe474f6222e5676e04bcf...           Elisay   
82  99b698fc3bfd04673cce4d4fc2d70823b92935b2b6378f... 

In [ ]:
# create global variable caching universalis requests
# update individual prices for all items in crafting tree
# for a top-level craft, get the min HQ unit sell price on Faerie
# for all other items, get the first quartile unit price on Aether (by mass not listings)
# then loop through again to add recipe costs

# Take the lower of (total mats price) * (recipe yield) and crafted item price, iterating through the recipes
# If the mats price is lower, use that going forwards in the calculations
# Then the last step is to compare total material buy price to item sell price